In [1]:
Documentation 
Question 1 Which stochastic optimiser did you choose for Task 3

Simmulated Annealing

Question 2 Why did you choose this optimiser?

There were multiple variables making the optimisation complex

Question 3 What constraint handling method(s) are you using and why?

SyntaxError: invalid syntax (<ipython-input-1-799f4078e3cc>, line 2)

In [2]:
import numpy as np
import matplotlib as mpl
import math
import random

In [3]:
# Question 1
#: the diameter of the wire (x0), 
#the mean of the diameter of coil(x1)
#and the number of active coils (x2)

In [4]:
arr_X = np.array([1, 2, 3])

In [5]:
#implementation of f(x)

In [6]:
def f(x, fCount=[0]):
    fun = float(x[0] ** 2)*x[1]*(2 + x[2])
#     print(fun)
    fCount[0]+=1
    return [fun, fCount[0]]

def g1(x, g1Count=[0]):
    g1Count[0]+=1
    fun = float(1 - (((x[1]**3)*x[2])/(71785*(x[0]**4))))
    if ( fun <= 0) :
        return [True, g1Count[0]]
    else:
        return [False, g1Count[0]]
    
def g2(x, g2Count=[0]):
    fun = (   (((4*x[1]**2) - (x[0]*x[1]))/(12566*((x[1]*(x[0]**3))-(x[0]**4)))) + (1/(5108*(x[0]**2)))  - 1 )
    g2Count[0]+=1
    if fun <= 0:
        return[True, g2Count[0]]
    else:
        return[False, g2Count[0]]
    
def g3(x, g3Count=[0]):
    fun = (1 - (140.45*x[0])/((x[1]**2)*x[2]))
    g3Count[0]+=1
    if fun <= 0:   
        return[True, g3Count[0]]
    else:
        return[False, g3Count[0]]
    
def g4(x, g4Count=[0]):
    fun = (  ((x[0] + x[1])/1.5)-1  )
    g4Count[0]+=1
    if fun <= 0:   
        return[True, g4Count[0]]
    else:
        return[False, g4Count[0]]
    
    
print(f(arr_X))
print(f(arr_X))
print(f(arr_X))

print (g1(arr_X))
print (g1(arr_X))
print (g1(arr_X))

print (g2(arr_X))
print (g2(arr_X))
print (g2(arr_X))

print (g3(arr_X))
print (g3(arr_X))
print (g3(arr_X))

print (g4(arr_X))
print (g4(arr_X))
print (g4(arr_X))  

[10.0, 1]
[10.0, 2]
[10.0, 3]
[False, 1]
[False, 2]
[False, 3]
[True, 1]
[True, 2]
[True, 3]
[True, 1]
[True, 2]
[True, 3]
[False, 1]
[False, 2]
[False, 3]


In [7]:
print(f(arr_X))
print(f(arr_X))
print(f(arr_X))

[10.0, 4]
[10.0, 5]
[10.0, 6]


In [8]:
#Question 2

In [9]:
wire_lower = 0.005 #lower bound for diameter of the wire
wire_upper = 2 #upper bound for diameter of the wire

coil_lower = 0.25 #lower bound for diameter of coil
coil_upper = 1.3 #upper bound for diameter of the coil

coil_num_lower = 2 #lower bound for number of active coils
coil_num_upper = 15 #upper bound for number of active coils

In [10]:
def generate_random_array():
    #generate random number 
    x0 = np.random.uniform(low=0.05, high=2) #hightest + 1
    x1 = np.random.uniform(low=0.25, high=1.3) #hightest + 1
    x2 = np.random.uniform(low=2, high=15) #hightest + 1
    
    return np.array([x0, x1, x2])
    

In [11]:
def is_feasible(array):
    feasible = False
    g1_arr=g1(array)
    g2_arr=g2(array)
    g3_arr=g3(array)
    g4_arr=g4(array)

#     print(g1_arr)
#     print(g2_arr)
#     print(g3_arr)
#     print(g4_arr)

    if g1_arr[0] and g2_arr[0] and g3_arr[0] and g4_arr[0] :
        feasible = True
    else:
        feasible = False
    return feasible
    

In [12]:
def random_search(best_fx, best_x):
    
    arr_x = generate_random_array()
    #print(arr_x)
    #print(is_feasible(arr_x))
    if is_feasible(arr_x):
        fx = f(arr_x)
        if fx < best_fx:
            best_x = np.array([arr_x[0], arr_x[1], arr_x[2]])
            print("best replaced")
            
    return best_x

In [13]:
def generate_feasible_random_array():
    array = generate_random_array()
    feasible = is_feasible(array)
    while not feasible:
        array = generate_random_array()
        feasible = is_feasible(array)
    return array

In [29]:
best_x = generate_random_array()
best_fx = f(best_x)

for x in range(0, 3000):
    best_x = random_search(best_fx, best_x)
print(best_x)
print(best_fx)
#print(arr_X)

#myArray = np.array(arrX)
#print(myArray)

# test = np.array([1, 2, 3])
#print (g1(arr_X))
# # #def searching(arr_X, best_X, best_fx):



best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
[ 0.06579861  0.55531866 12.83916709]
[13.901159259527285, 21104]


In [25]:
def distort(array):
    x0 = array[0]
    x1 = array[1]
    x2 = array[2]
    
    x0_new = 1000 #not feasible to begin with
    x1_new = 1000 #not feasible to begin with
    x2_new = 1000 #not feasible to begin with
    
    while not is_feasible(np.array([x0_new, x1, x2])):
        x0_new = np.random.normal(x0, 1)
#     print("i did go here")
    
    while not is_feasible(np.array([x0_new, x1_new, x2])):
        x1_new = np.random.normal(x1,1)
#     print("i did go here")
    
    while not is_feasible(np.array([x0_new, x1_new, x2_new])):
        x2_new = np.random.normal(x2,1)
#     print("i did go here")
   # print(str(x0_new) + " " + str(x1_new) + " " + str(x2_new))
        
    return [x0_new, x1_new, x2_new]

In [16]:
start_values_array = generate_feasible_random_array()
print(start_values_array)
print(distort(start_values_array))

[ 0.06217621  0.6116375  13.77427171]
[0.07643947804634851, 0.6641843061020776, 15.252171576092781]


In [17]:
#Question 3

#aim 
#to get minimum fx
#aka the energy
#when system goes through change of configuratioj we have to calculate delta e, change in energy

#how do we chnge te configuration - randomly change the configuration c, making a move

#temperature start with max temp and then slowly decreases to t min

#for every value of temperature 
    #compute energy 
    #change config and get energy of new config
    #calculate change in energy 
    #if change in e is -ve
        #accept the move
    #else if
        #the closer we get to the end of the cycle, the less likely we are to accept bad decisions
        #compute prob e^ change in e/temperature > rand(0,1)
        #accept the move
#step 1 random 

#step 2 more careful

In [27]:
start_values_array = generate_feasible_random_array()
print("lets start at random " + str(start_values_array))
best_fx = f(start_values_array)
print("best fx so far " + str(best_fx))

#do for number of iterations
temp_max = 3000
temp_min = 0
for t in range(temp_max, temp_min, -1):
#     print(t)
    
    #new config
#     print("finding neighboring set of values to compare")
    new_neighbours = distort(start_values_array)
#     print(new_neighbours)
    new_fx = f(new_neighbours)
    
    #calculae change in energy
#     print("finding delta e")
    delta_fx = best_fx[0] - new_fx[0]
    
    if delta_fx < 0:
        print("new delta e in the house by merit " + str(t) +" " + str(new_fx))
        best_fx = new_fx
        #print("I was bested")
    elif (math.exp(delta_fx/t) < random.uniform(0, 1)):
        print("new delta e in the house by probability" + str(t) +" " + str(new_fx))
        best_fx = new_fx
        #print(math.exp(delta_fx/t))
        
print("finished")
    

lets start at random [0.06916575 0.90657619 3.45982033]
best fx so far [0.023679083789688932, 18087]
new delta e in the house by merit 3000 [0.030410400385094037, 18088]
new delta e in the house by merit 2999 [0.043174058847020635, 18089]
new delta e in the house by merit 2988 [0.04609984727503637, 18100]
new delta e in the house by merit 2921 [0.04660377221640052, 18167]
new delta e in the house by merit 2903 [0.047047444578143534, 18185]
new delta e in the house by merit 2877 [0.047194616546797706, 18211]
new delta e in the house by merit 2844 [0.05176327340922674, 18244]
new delta e in the house by merit 2711 [0.05862427195363002, 18377]
finished
